In [19]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras
import pandas as pd
# Bibliotecas Auxiliares
import seaborn as sns
import numpy as np
import collections
import matplotlib.pyplot as plt
import nest_asyncio
nest_asyncio.apply()
import tensorflow_federated as tff
np.random.seed(0)
from collections import Counter

import datetime;


from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow import keras

import collections
import functools
import os
import time
from scipy import stats

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter


print(tf.__version__)

/bin/bash: /home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2.10.0


In [2]:
# y_test     = Array with real values
# yhat_probs = Array with predicted values
def printMetrics(y_test,yhat_probs):
    # predict crisp classes for test set deprecated
    #yhat_classes = model.predict_classes(X_test, verbose=0)
    #yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_classes = yhat_probs.round()
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(y_test, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = roc_auc_score(y_test, yhat_probs)
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("\Confusion Matrix")
    matrix = confusion_matrix(y_test, yhat_classes)
    print(matrix)
    
    results = dict()
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1_score'] = f1
    results['cohen_kappa_score'] = kappa
    results['roc_auc_score'] = auc
    return results

def showGlobalMetrics(metrics):
    accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score = 0,0,0,0,0,0
    for metric in metrics:
        accuracy = accuracy + metric['accuracy']
        precision = precision + metric['precision']
        recall = recall + metric['recall']
        f1_score = f1_score + metric['f1_score']
        cohen_kappa_score = cohen_kappa_score + metric['cohen_kappa_score']
        roc_auc_score = roc_auc_score + metric['roc_auc_score']
        
    # mean
    size = len(metrics)
    print(size)
    accuracy = accuracy / size
    precision = precision / size
    recall = recall / size
    f1_score = f1_score / size
    cohen_kappa_score = cohen_kappa_score / size
    roc_auc_score = roc_auc_score / size
    
    #show:\
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    print("f1_score: ",f1_score)
    print("cohen_kappa_score: ",cohen_kappa_score)
    print("roc_auc_score: ",roc_auc_score)
    
def create_dataset_time_series_with_one_output(X, y, window_time_steps=1, shift_step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - window_time_steps, shift_step):
        v = X.iloc[i:(i + window_time_steps)].values
        labels = y.iloc[i: i + window_time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
        
    if len(y.columns) == 1:
        return np.array(Xs), np.array(ys).reshape(-1, 1)
    else:
        return np.array(Xs), np.array(ys).reshape(-1, len(y.columns))

In [3]:
# configs
EPOCHS = 30
NUM_EPOCHS = EPOCHS
BATCH_SIZE = 32

TIME_SERIES_SIZE = 4   # Determines the window size. Ex (4,9)
TIME_STEP_SHIFT  = 1   # Determines specifies the number of steps to move the window forward at each iteration.

baseFolder = "../data_2019_processed/"

# selected features
inputFeatures = ["activity","location","day_of_week",
                 "light","phone_lock","proximity",
                 "sound","time_to_next_alarm", "minutes_day"]
# outputs
outputClasses = ["awake","asleep"]
#outputClasses = ["class"]

In [4]:
# test data comprising 25% of the data. It must be fixed to all models being evaluated
X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized.csv")

# input folder
inputFolders = baseFolder

# client datasets used on the training process
dsTrain =  ['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
            '0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
            '2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
            '2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
            #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
            '7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
            'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
            'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
            'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
            'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
            'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
            'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', 
            'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
            'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
            'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
            'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
            'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
            'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
            'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
            #['PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc'], 
            'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
            #['rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw'], 
            #['RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI'], 
            'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4'] 
            #['VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is'], 
            #['Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw'], 
            #['XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA'], 
            #['YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw'], 
            #['ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM'], 
            #['ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']]
            
dsTrain = ['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
            '0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
            '2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
            '2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
            #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
            '7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
            'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
            'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
            'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
            'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
            'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
            'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', 
            'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
            'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
            'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
            'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
            'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
            'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
            'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
            'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', 
            'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
            'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
            'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
            'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4', 
            'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
            'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
            'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
            'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
            'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
            'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']




In [5]:
# load cliend data
clientList = []

# balancing classes
#oversample = RandomOverSampler(sampling_strategy='auto') #minority
#undersample = RandomUnderSampler(sampling_strategy='majority')

for i in range(0,len(dsTrain)):
    print (i," ", str(inputFolders)+"student_"+dsTrain[i]+"_numeric.csv") #_numeric
    # load client data
    dataset = pd.read_csv(inputFolders+"student_"+dsTrain[i]+"_numeric.csv")
    
    # print(dataset)
    y_train = dataset['class'].copy()
    
    # does not add datasets that dont have instances from both classes
    if y_train.sum() != 0 and (y_train.sum() != len(y_train)):
        X_train = dataset
        # balancing classes
        #X_train, y_train = oversample.fit_resample(dataset, y_train)
        #X_train, y_train = undersample.fit_resample(dataset, y_train)
        
        X_train['class'] = y_train
        
        clientList.append(X_train)
        
print("Total",(len(clientList)))

0   ../data_2019_processed/student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs_numeric.csv
1   ../data_2019_processed/student_0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA_numeric.csv
2   ../data_2019_processed/student_2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0_numeric.csv
3   ../data_2019_processed/student_2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys_numeric.csv
4   ../data_2019_processed/student_7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA_numeric.csv
5   ../data_2019_processed/student_a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4_numeric.csv
6   ../data_2019_processed/student_ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc_numeric.csv
7   ../data_2019_processed/student_Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U_numeric.csv
8   ../data_2019_processed/student_CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA_numeric.csv
9   ../data_2019_processed/student_DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc_numeric.csv
10   ../data_2019_processed/student_DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA_numeric.csv
11   ../

In [6]:
# undestand the dataset by looking on their infos
print(X_test.info())

X_test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136286 entries, 0 to 136285
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  float64
 1   location            136286 non-null  float64
 2   timestamp           136286 non-null  float64
 3   day_of_week         136286 non-null  float64
 4   light               136286 non-null  float64
 5   phone_lock          136286 non-null  float64
 6   proximity           136286 non-null  float64
 7   sound               136286 non-null  float64
 8   time_to_next_alarm  136286 non-null  float64
 9   minutes_day         136286 non-null  float64
 10  timestamp_text      136286 non-null  object 
 11  class               136286 non-null  object 
dtypes: float64(10), object(2)
memory usage: 12.5+ MB
None


,activity,location,timestamp,day_of_week,light,phone_lock,proximity,sound,time_to_next_alarm,minutes_day,timestamp_text,class
0,0.00,1.0,0.000647,1.000000,0.000000,0.0,1.0,0.655565,0.906151,0.115358,2018-05-15 02:46:57,asleep
1,0.25,1.0,0.000647,1.000000,0.000000,0.0,1.0,0.567296,0.906052,0.116053,2018-05-15 02:47:27,asleep
2,0.25,1.0,0.000647,1.000000,0.000000,0.0,1.0,0.567296,0.906052,0.116053,2018-05-15 02:47:57,asleep
3,0.25,1.0,0.000647,1.000000,0.000000,0.0,1.0,0.660604,0.905952,0.116748,2018-05-15 02:48:28,asleep
4,0.25,1.0,0.000647,1.000000,0.000000,0.0,1.0,0.660604,0.905952,0.116748,2018-05-15 02:48:57,asleep
...,...,...,...,...,...,...,...,...,...,...,...,...
136281,0.25,1.0,0.007015,0.166667,0.000056,1.0,1.0,0.000000,0.000099,0.510076,2018-06-13 12:14:37,awake
136282,0.25,1.0,0.007015,0.166667,0.000078,1.0,1.0,0.000000,0.000694,0.512856,2018-06-13 12:18:08,awake
136283,0.25,1.0,0.007015,0.166667,0.000078,1.0,1.0,0.000000,0.000595,0.513551,2018-06-13 12:19:08,awake
136284,0.25,1.0,0.007016,0.166667,0.000085,1.0,1.0,0.000000,0.000595,0.513551,2018-06-13 12:19:38,awake


In [7]:
# one-hot encoding function
def transform_output_nominal_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data['awake']
    dataset['asleep'] = one_hot_encoded_data['asleep']
    
    return dataset

# one-hot encoding function
def transform_output_numerical_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data[0]
    dataset['asleep'] = one_hot_encoded_data[1]
    
    return dataset

# transform output to one_hot_encoding for the testing dataset
X_test = transform_output_nominal_class_into_one_hot_encoding(X_test)

# transform output to one_hot_encoding for the input dataset
for i in range(0,len(clientList)):
    clientList[i] = transform_output_numerical_class_into_one_hot_encoding(clientList[i])
    #print (clientList[i])

In [8]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136286 entries, 0 to 136285
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  float64
 1   location            136286 non-null  float64
 2   timestamp           136286 non-null  float64
 3   day_of_week         136286 non-null  float64
 4   light               136286 non-null  float64
 5   phone_lock          136286 non-null  float64
 6   proximity           136286 non-null  float64
 7   sound               136286 non-null  float64
 8   time_to_next_alarm  136286 non-null  float64
 9   minutes_day         136286 non-null  float64
 10  timestamp_text      136286 non-null  object 
 11  class               136286 non-null  object 
 12  awake               136286 non-null  uint8  
 13  asleep              136286 non-null  uint8  
dtypes: float64(10), object(2), uint8(2)
memory usage: 12.7+ MB


In [9]:
def transform_data_type(dataframe):
    
    # transform inputs
    for column in inputFeatures:
        dataframe[column] = dataframe[column].astype('float32')
    
    # transform outputs
    for column in outputClasses:
        dataframe[column] = dataframe[column].astype('float32')
    
    return dataframe

# transforms the data
X_test = transform_data_type(X_test)

X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136286 entries, 0 to 136285
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  float32
 1   location            136286 non-null  float32
 2   timestamp           136286 non-null  float64
 3   day_of_week         136286 non-null  float32
 4   light               136286 non-null  float32
 5   phone_lock          136286 non-null  float32
 6   proximity           136286 non-null  float32
 7   sound               136286 non-null  float32
 8   time_to_next_alarm  136286 non-null  float32
 9   minutes_day         136286 non-null  float32
 10  timestamp_text      136286 non-null  object 
 11  class               136286 non-null  object 
 12  awake               136286 non-null  float32
 13  asleep              136286 non-null  float32
dtypes: float32(11), float64(1), object(2)
memory usage: 8.8+ MB


In [10]:
# selects the data to train and test
X_test_data = X_test[inputFeatures]
y_test_label = X_test[outputClasses]

# transform to time series
X_test_data, y_test_label = create_dataset_time_series_with_one_output(    #timestamp
    X_test_data, 
    y_test_label, 
    TIME_SERIES_SIZE, 
    TIME_STEP_SHIFT
)

# transtorm data to tensor slices
client_test_dataset = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_label))
#client_test_dataset = tf.data.Dataset.from_tensor_slices((X_test_data.values, y_test_label.values))

#client_test_dataset = client_test_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE, drop_remainder=True)
client_test_dataset = client_test_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE)

print(client_test_dataset.element_spec)
client_test_dataset

2023-03-04 13:07:50.117606: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-04 13:07:50.195765: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-04 13:07:50.196311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-04 13:07:50.198322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

(TensorSpec(shape=(None, 4, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))


ut NUMA support.
2023-03-04 13:07:50.202816: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-04 13:07:50.203114: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-04 13:07:53.262694: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-04 13:07:53.264719: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-04 13:07:53.264755: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1700

<BatchDataset element_spec=(TensorSpec(shape=(None, 4, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [11]:
federated_training_data = []
# transform the data
for i in range(0,len(clientList)):
    # selects the data to train and test
    data   = clientList[i][inputFeatures]
    labels = clientList[i][outputClasses]
    # transform data to float32
    #data = transform_data_type(data)

    # transform to time series
    data, labels = create_dataset_time_series_with_one_output(    #timestamp
        data, 
        labels, 
        TIME_SERIES_SIZE, 
        TIME_STEP_SHIFT
    )

    # transform the data to tensor slices
    client_train_dataset = tf.data.Dataset.from_tensor_slices((data, labels))
    #client_train_dataset = tf.data.Dataset.from_tensor_slices((data.values, labels.values))
    # apply the configs
    client_train_dataset = client_train_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE)
    # transform the data to
    federated_training_data.append(client_train_dataset)

In [12]:
def create_keras_model():
    model = keras.Sequential()
    model.add(LSTM(128,input_shape=(4, 9,))) #return_sequences=True, 
    #model.add(LSTM(128,input_shape=(2, 9,)))
    #model.add(keras.layers.Dropout(rate=0.5))
    #model.add(LSTM(64,activation="tanh"))
    model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.Dense(2, activation='softmax'))#softmax,sigmoid
    
    return model

In [13]:
keras_model = create_keras_model()
#keras_model.summary()
keras_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               70656     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 70,914
Trainable params: 70,914
Non-trainable params: 0
_________________________________________________________________


In [14]:
def model_fn():
    # We _must_ create a new model here, and _not_ capture it from an external
    # scope. TFF will call this within different graph contexts.
    #keras_model = create_keras_model()
    keras_model = create_keras_model()
    return tff.learning.from_keras_model(
      keras_model,
      input_spec=client_train_dataset.element_spec,
      loss=tf.keras.losses.CategoricalCrossentropy(), #BinaryCrossentropy
      metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Precision()])

In [15]:
fed_avg_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),#client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [16]:
print(fed_avg_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[9,512],
      float32[128,512],
      float32[512],
      float32[128,2],
      float32[2]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64
  >
>@SERVER)


In [17]:
state = fed_avg_process.initialize()

In [20]:
current_time = datetime.datetime.now()
time_stamp = current_time.timestamp()
print("Start timestamp:", time_stamp,current_time)
    
result = fed_avg_process.next(state, federated_training_data[0:10])
state = result.state
metrics = result.metrics
print('round  1, metrics={}'.format(metrics))


current_time = datetime.datetime.now()
time_stamp = current_time.timestamp()
print("End timestamp:", time_stamp,current_time)

Start timestamp: 1677939801.527352 2023-03-04 14:23:21.527352


2023-03-04 14:23:32.072025: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.83657944), ('precision', 0.83527976), ('loss', 0.378862), ('num_examples', 5728980), ('num_batches', 179034)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
End timestamp: 1677944374.697553 2023-03-04 15:39:34.697553


In [21]:
#def keras_evaluate(state, round_num):
    # Take our global model weights and push them back into a Keras model to
    # use its standard `.evaluate()` method.
keras_model = create_keras_model()


keras_model.compile(
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Precision()])

# get neural network weights
weights = fed_avg_process.get_model_weights(state)
weights.assign_weights_to(keras_model)


#tttt.model.assign_weights_to(keras_model)
yhat_probs = keras_model.predict(X_test_data)

maxX = yhat_probs.max()
minX = yhat_probs.min()
avgX = yhat_probs.mean()

print(maxX,minX,avgX)
print(yhat_probs)
# predict crisp classes for test set deprecated
#printMetrics(y_test,yhat_probs)
test = list()

xx = yhat_probs.round()

y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

print('')
print('awake')
res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

2023-03-04 15:39:35.133146: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19624608 exceeds 10% of free system memory.
2023-03-04 15:39:35.277175: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19624608 exceeds 10% of free system memory.
2023-03-04 15:39:38.155643: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201


4259/4259 [==============================] - 13s 2ms/step
0.9838154 0.016184678 0.5
[[0.8782575  0.12174246]
 [0.8813447  0.11865527]
 [0.87621903 0.12378095]
 ...
 [0.8585065  0.14149348]
 [0.85854703 0.14145297]
 [0.85858154 0.1414184 ]]

awake
Accuracy: 0.709881
Precision: 0.707953
Recall: 0.998096
F1 score: 0.828352
Cohens kappa: 0.042904
ROC AUC: 0.515523
\Confusion Matrix
[[ 1341 39356]
 [  182 95403]]

asleep
Accuracy: 0.711547
Precision: 0.879186
Recall: 0.033090
F1 score: 0.063779
Cohens kappa: 0.043168
ROC AUC: 0.515585
\Confusion Matrix
[[95632   184]
 [39127  1339]]

Global
2
accuracy:  0.7107138140033167
precision:  0.7935692665270488
recall:  0.5155927213976942
f1_score:  0.44606549299857856
cohen_kappa_score:  0.043036361729402284
roc_auc_score:  0.5155539818049104


In [22]:
y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

print('')
print('awake')
res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)


awake
Accuracy: 0.709881
Precision: 0.707953
Recall: 0.998096
F1 score: 0.828352
Cohens kappa: 0.042904
ROC AUC: 0.515523
\Confusion Matrix
[[ 1341 39356]
 [  182 95403]]

asleep
Accuracy: 0.711547
Precision: 0.879186
Recall: 0.033090
F1 score: 0.063779
Cohens kappa: 0.043168
ROC AUC: 0.515585
\Confusion Matrix
[[95632   184]
 [39127  1339]]

Global
4
accuracy:  0.7107138140033167
precision:  0.7935692665270488
recall:  0.5155927213976941
f1_score:  0.44606549299857856
cohen_kappa_score:  0.043036361729402284
roc_auc_score:  0.5155539818049104


In [23]:
current_time = datetime.datetime.now()
time_stamp = current_time.timestamp()
print("Start timestamp:", time_stamp,current_time)

result = fed_avg_process.next(state, federated_training_data[10:19])
state = result.state
metrics = result.metrics
print('round  2, metrics={}'.format(metrics))

current_time = datetime.datetime.now()
time_stamp = current_time.timestamp()
print("End timestamp:", time_stamp,current_time)

Start timestamp: 1677944392.439174 2023-03-04 15:39:52.439174
round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8570261), ('precision', 0.8558325), ('loss', 0.3386606), ('num_examples', 6152940), ('num_batches', 192283)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
End timestamp: 1677949240.372085 2023-03-04 17:00:40.372085


In [24]:
#def keras_evaluate(state, round_num):
    # Take our global model weights and push them back into a Keras model to
    # use its standard `.evaluate()` method.
keras_model = create_keras_model()


keras_model.compile(
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=[tf.keras.metrics.CategoricalAccuracy()])

# get neural network weights
weights = fed_avg_process.get_model_weights(state)
weights.assign_weights_to(keras_model)


#tttt.model.assign_weights_to(keras_model)
yhat_probs = keras_model.predict(X_test_data)

maxX = yhat_probs.max()
minX = yhat_probs.min()
avgX = yhat_probs.mean()

print(maxX,minX,avgX)
print(yhat_probs)
# predict crisp classes for test set deprecated
#printMetrics(y_test,yhat_probs)
test = list()

xx = yhat_probs.round()

y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

print('')
print('awake')
res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

2023-03-04 17:00:40.604182: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19624608 exceeds 10% of free system memory.


4259/4259 [==============================] - 10s 2ms/step
0.9864123 0.013587741 0.5
[[0.8403469  0.15965317]
 [0.8522035  0.14779651]
 [0.84086245 0.15913753]
 ...
 [0.87140274 0.12859732]
 [0.8715912  0.12840877]
 [0.87172055 0.1282794 ]]

awake
Accuracy: 0.718796
Precision: 0.730966
Recall: 0.947973
F1 score: 0.825445
Cohens kappa: 0.160668
ROC AUC: 0.564251
\Confusion Matrix
[[ 7347 33350]
 [ 4973 90612]]

asleep
Accuracy: 0.720124
Precision: 0.594318
Recall: 0.180942
F1 score: 0.277422
Cohens kappa: 0.161155
ROC AUC: 0.564390
\Confusion Matrix
[[90818  4998]
 [33144  7322]]

Global
2
accuracy:  0.7194603836163249
precision:  0.6626420614968517
recall:  0.5644575168606237
f1_score:  0.5514335824069798
cohen_kappa_score:  0.16091158798916355
roc_auc_score:  0.5643204582451413


In [25]:
for i in range(0,2):
    
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"Start timestamp:", time_stamp,current_time)

    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))
    
    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"End timestamp:", time_stamp,current_time)
    print('')
    print('')

2 Start timestamp: 1677949252.178667 2023-03-04 17:00:52.178667
round  2-4, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.85537136), ('precision', 0.85414106), ('loss', 0.34289375), ('num_examples', 11881920), ('num_batches', 371317)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 10s 2ms/step
0.985007 0.014993022 0.5
[[0.84831166 0.15168834]
 [0.86372554 0.13627444]
 [0.8525518  0.1474482 ]
 ...
 [0.8612799  0.1387201 ]
 [0.8614126  0.13858742]
 [0.8615367  0.13846333]]

awake
Accuracy: 0.731498
Precision: 0.741215
Recall: 0.948245
F1 score: 0.832045
Cohens kappa: 0.210275
ROC AUC: 0.585335
\Confusion Matrix
[[ 9052 31645]
 [ 4947 90638]]

asleep
Accuracy: 0.732841
Precision: 0.644903
Recall: 0.223101
F1 score: 0.331516
Cohens kappa: 0.211100
ROC AUC: 0.585610
\Confusion Matrix
[

KeyboardInterrupt: 

In [ ]:
for i in range(2,4):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"Start timestamp:", time_stamp,current_time)

    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"End timestamp:", time_stamp,current_time)
    print('')
    print('')

In [ ]:
import datetime;
  
current_time = datetime.datetime.now()
  
time_stamp = current_time.timestamp()
print("Start timestamp:-", time_stamp,current_time)

In [ ]:
for i in range(4,6):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"Start timestamp:", time_stamp,current_time)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"End timestamp:", time_stamp,current_time)
    print('')
    print('')

In [27]:
for i in range(6,7):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"Start timestamp:", time_stamp,current_time)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"End timestamp:", time_stamp,current_time)
    print('')
    print('')

14 Start timestamp: 1677876295.287352 2023-03-03 20:44:55.287352
round  14-16, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.84840226), ('precision', 0.8471487), ('loss', 0.3568586), ('num_examples', 3960640), ('num_batches', 123779)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 9s 2ms/step
0.9879844 0.012015647 0.5
[[0.8380231  0.16197689]
 [0.8495376  0.15046233]
 [0.83833677 0.16166326]
 ...
 [0.8180887  0.18191127]
 [0.81829    0.18171003]
 [0.81843585 0.18156414]]

awake
Accuracy: 0.730918
Precision: 0.741772
Recall: 0.945504
F1 score: 0.831338
Cohens kappa: 0.211620
ROC AUC: 0.586212
\Confusion Matrix
[[ 9235 31462]
 [ 5209 90376]]

asleep
Accuracy: 0.732202
Precision: 0.637427
Recall: 0.227524
F1 score: 0.335349
Cohens kappa: 0.212300
ROC AUC: 0.586434
\Confusion Matrix
[

KeyboardInterrupt: 

In [26]:
for i in range(7,8):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"Start timestamp:", time_stamp,current_time)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"End timestamp:", time_stamp,current_time)
    print('')
    print('')

16 Start timestamp: 1677961381.306848 2023-03-04 20:23:01.306848
round  16-18, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8641499), ('precision', 0.86294705), ('loss', 0.32803243), ('num_examples', 11881920), ('num_batches', 371317)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 10s 2ms/step
0.9891698 0.010830211 0.5
[[0.87159854 0.12840146]
 [0.8845958  0.11540413]
 [0.8761793  0.12382074]
 ...
 [0.8562591  0.14374092]
 [0.8564318  0.1435682 ]
 [0.85659146 0.14340849]]

awake
Accuracy: 0.729693
Precision: 0.729755
Recall: 0.976063
F1 score: 0.835127
Cohens kappa: 0.164562
ROC AUC: 0.563553
\Confusion Matrix
[[ 6147 34550]
 [ 2288 93297]]

asleep
Accuracy: 0.731124
Precision: 0.726615
Recall: 0.151460
F1 score: 0.250670
Cohens kappa: 0.165152
ROC AUC: 0.563697
\Confusion Matri

In [27]:
for i in range(8,16):
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"Start timestamp:", time_stamp,current_time)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}-{}, metrics={}'.format(i*2+2,i*2+4,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    #tttt.model.assign_weights_to(keras_model)
    yhat_probs = keras_model.predict(X_test_data)

    maxX = yhat_probs.max()
    minX = yhat_probs.min()
    avgX = yhat_probs.mean()

    print(maxX,minX,avgX)
    print(yhat_probs)
    # predict crisp classes for test set deprecated
    #printMetrics(y_test,yhat_probs)
    test = list()

    xx = yhat_probs.round()

    y_test2 = pd.DataFrame(data=xx,columns=['awake','asleep']) 
    y_test_label_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

    print('')
    print('awake')
    res = printMetrics(y_test_label_label['awake'],y_test2['awake'])
    test.append(res)
    print('')
    print('asleep')
    res = printMetrics(y_test_label_label['asleep'],y_test2['asleep'])
    test.append(res)
    print('')
    print('Global')
    showGlobalMetrics(test)
    print('')
    current_time = datetime.datetime.now()
    time_stamp = current_time.timestamp()
    print(i*2+2,"End timestamp:", time_stamp,current_time)
    print('')
    print('')

18 Start timestamp: 1677977406.098739 2023-03-05 00:50:06.098739
round  18-20, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8735254), ('precision', 0.87233984), ('loss', 0.310535), ('num_examples', 11881920), ('num_batches', 371317)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])

4259/4259 [==============================] - 10s 2ms/step
0.99354 0.00646004 0.5
[[0.8905278  0.10947224]
 [0.9025497  0.09745031]
 [0.8948299  0.10517015]
 ...
 [0.8444267  0.1555733 ]
 [0.8446905  0.15530951]
 [0.84497166 0.15502836]]

awake
Accuracy: 0.725334
Precision: 0.725111
Recall: 0.979850
F1 score: 0.833451
Cohens kappa: 0.140746
ROC AUC: 0.553701
\Confusion Matrix
[[ 5191 35506]
 [ 1926 93659]]

asleep
Accuracy: 0.726721
Precision: 0.726430
Recall: 0.127762
F1 score: 0.217304
Cohens kappa: 0.141006
ROC AUC: 0.553721
\Confusion Matrix
[[9

KeyboardInterrupt: 